# Texture Atlas Builder
### By Beherith (mysterme@gmail.com)
Run the first cell before you do anything

In [18]:
# INITIALIZATION YOU MUST START BY RUNNING THIS CELL!
# This makes an atlas out of a set of arbitrarily sized images. (c) Beherith mysterme@gmail.com

from PIL import Image
import os
import math
import fnmatch



def LoadImgList(l, scale = 1, padding = 0):
    res = []
    for ifn in l:
        i = Image.open(os.path.join(bar_sdd_path, ifn))
        if scale >1 :
            i = i.resize((i.size[0]//scale, i.size[1]//scale), Image.BICUBIC) # could also be Image.LANCZOS
        res.append([ifn, i.size[0] + padding, i.size[1]+padding,i])
    res = sorted(res, key = lambda x:x[0], reverse = True) #filenames last
    res = sorted(res, key = lambda x:x[2], reverse = True) #then height
    res = sorted(res, key = lambda x:x[1], reverse = True) #primarily width
    return res

def getNearestPowerOfTwoDims(imglist):
    totalsize = 0
    maxw = 0
    maxh = 0
    for img in imglist:
        maxw = max(maxw, img[1])
        maxh = max(maxh, img[2])
        totalsize += img[1]*img[2]
    side = True
    print(maxh,maxw)
    maxw = 1 << math.ceil(math.log(maxw,2))
    maxh = 1 << math.ceil(math.log(maxh,2))
    while (maxw * maxh< totalsize):
        if side:
            maxw = min(maxw+1024, maxw*2)
        else:
            maxh = min(maxh+1024, maxh*2)
        side = not side
    print(f"Atlas size for {len(imglist)} images is at least {maxw} x {maxh} for {totalsize} pixels.")
    return maxw, maxh

class Node:
    def __init__(self, x,y,w,h,p):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.down = None
        self.right = None
        self.used = False
        self.id = None
        self.pos = p
    def findNode(self, node, w, h, d = 0):
        #print (f"{'-'*d} {node.pos} Find {node.id} with size {node.w}x{node.h} fits w{w}xh{h}?")
        if node.used:
            return self.findNode(node.right,w,h,d+1) or self.findNode(node.down,w,h,d+1)
        elif node.w >= w and node.h >= h:
            return node
        else:
            return False
    def splitNode(self, node,w,h):
        node.used=True
        node.down = Node(node.x, node.y + h, w, node.h - h,'d')
        node.right = Node(node.x +w, node.y, node.w -w, node.h,'r')
    def fit(self, img, padding):
        node = self.findNode( self,img[1], img[2])
        node.splitNode(node, img[1], img[2])
        node.id = img[0]
        node.w = img[1] - padding
        node.h = img[2] - padding
        #print (f"Fit {node.id} to {node.x} : {node.y}")
        return node


def writeLua(basename, rootpath, coords, aw, ah):
    luafile = open(os.path.join(bar_sdd_path, rootpath, basename+".lua"),'w')
    
    luafile.write("-- This file has been automatically generated by texture_atlas_builder.ipynb\n")
    luafile.write("-- Do not edit this file!\n")
    luafile.write("-- Note that the UV coordinates are unpadded, if you must avoid bleed then pad it with 0.5/atlas size \n")
    luafile.write('local atlas = {\n')
    luafile.write('\tatlasimage = "%s/%s.dds",\n'%(rootpath, basename))
    luafile.write('\twidth = %d,\n'%(aw))
    luafile.write('\theight = %d,\n'%(ah))
    luafile.write('\tflip = function(t) for k,v in pairs(t) do if type(v) == "table" then v[3], v[4] = 1.0 - v[3], 1.0 - v[4] end end end ,\n')
    luafile.write('\tpad = function(t,p) for k,v in pairs(t) do if type(v) == "table" then p = p or 0.5; local px,py = p/t.width, p/t.height; v[1], v[2], v[3], v[4] = v[1] + px, v[2]-px, v[3] + py, v[4] - py end end end ,\n')
    luafile.write('\tgetUVCoords = function(t, name) if t[name] then return t[name][1], t[name][2], t[name][3], t[name][4] else return 0,1,0,1 end end ,\n')

    for fname in sorted(coords.keys()):
        luafile.write('\t["%s"] = {%s}, \n'%(fname, ','.join(map(str,coords[fname]))))
    luafile.write('\n}\nreturn atlas\n')
    luafile.close()

def MakeAtlas(name = "my_atlas", sourcedir = "" , pattern = "*.png", gamedir = "", hasAlpha = True, bgColor = (0,0,0,0), padding = 0, scale = 1, flipY = False, invertG = False):
    if bar_sdd_path == '' or os.path.exists(bar_sdd_path) == False:
        print("You MUST specify the bar_sdd_path correctly to point to your working copy of BAR in order to use this tool! You have specified:", bar_sdd_path)
        return
    fnamelist =[]
    for root, dir, files in os.walk(os.path.join(bar_sdd_path, sourcedir)):
        for file in fnmatch.filter(files, pattern):
            if file == name + ".tga": # try not to include self
                print ("Ignoring", file, "as its already an atlas")
            else:
                vfspath = root[root.find(sourcedir):].replace('\\','/')
                fnamelist.append(vfspath + '/' +  file)
    print (fnamelist)
    imglist = LoadImgList(fnamelist, scale,padding)
    aw, ah = getNearestPowerOfTwoDims(imglist)
    for im in imglist[:4]:
        print(im)
        
    oimg = Image.new("RGBA" if hasAlpha else "RGB", (aw,ah), color = bgColor)
    root = Node(0,0,aw,ah,'r')
    uvcoords = {}
    for img in imglist:
        node = root.fit(img, padding)
        oimg.paste(img[3], (node.x, node.y) )
        uvcoords[img[0]] = [(node.x)/aw,(node.x+node.w)/aw,(node.y)/ah,(node.y+node.h)/ah,node.w, node.h] #xXyYwh
        if flipY:
            uvcoords[img[0]] = [(node.x)/aw,(node.x+node.w)/aw,1.0- (node.y)/ah,1.0- (node.y+node.h)/ah,node.w, node.h] #xXyYwh
            
    oimg.save(os.path.join(bar_sdd_path, sourcedir, name + ".tga"))
    #oimg.show()
    writeLua(name, sourcedir, uvcoords, aw, ah)
    #"%~dp0nvtt_export.exe" --output "%%~nx.dds" --save-flip-y --mip-filter 0 --quality 3 --format bc3 "%%~x"
    if nvtt_export_path == "" or os.path.exists(nvtt_export_path) == False:
        print ("You MUST specify the correct nvtt_export_path to be able to create compressed .dds files of the atlas", nvtt_export_path)
        return
    ddscmd = f'""{nvtt_export_path}" --output "{os.path.join(bar_sdd_path, sourcedir, name)}.dds" {"--save-flip-y" if flipY else ""} --mip-filter 0 --quality 3 --format {"bc3" if hasAlpha else "bc1"} "{os.path.join(bar_sdd_path, sourcedir, name)}.tga""'
    print(ddscmd)
    os.system(ddscmd)
    #print (subprocess.check_output(ddscmd, shell = True))

print("All libraries loaded successfully")

All libraries loaded successfully


# MakeAtlas Options

- name: name of atlas lua and image file, will end up as myname.lua and myname.tga and myname.dds
- sourcedir: the directory to process. Will also include subdirectories
- pattern: Which image files should beprocessed, e.g "*.png". Uses fnmatch
- hasalpha (default yes), whether the source images have and use an alpha channel.
- bgColor (default black), what the empty background color should be, default (0,0,0,0)
- padding (default 0), how much of a padding border to leave between images, in pixels
- scale, default 1, what factor should all of the images be downscaled at. 1 means no scaling, 2 means halfscale, 3, is 1/3rd
- flipY : flip the result along the Y axis


## Configure Paths:

bar_sdd_path - point it to bar sdd dir. Please use / characters for dirs

nvtt_export_path -- point it to nvtt_export.exe, use / chars for dirs

In [ ]:
bar_sdd_path = "C:/Users/Peti/Documents/My Games/Spring/games/Beyond-All-Reason.sdd"
nvtt_export_path = "C:/Users/Peti/Documents/My Games/Spring/springrts_smf_compiler/NVTT_DragAndDropConvertToDDSTools/nvtt_export.exe"

MakeAtlas(name = "icon_atlas_full_small", sourcedir= "icons", pattern = "*.png", scale = 4, padding = 0)

### Decals GL4 Atlasses

In [ ]:

MakeAtlas(name = "decalsgl4_atlas_diffuse", sourcedir= "luaui/images/decals_gl4", pattern = "*_a.*")
MakeAtlas(name = "decalsgl4_atlas_normal", sourcedir= "luaui/images/decals_gl4", pattern = "*_n.*")

### Unit and feature AO Plates Atlas


In [ ]:

MakeAtlas(name = "unitaoplates_atlas", sourcedir= "unittextures/decals", pattern = "*_aoplane.dds")
MakeAtlas(name = "featureaoplates_atlas", sourcedir= "unittextures/decals_features", pattern = "*_aoplane.*")

# Icons and BuildPics

In [ ]:
MakeAtlas(name = "icon_atlas", sourcedir= "icons", pattern = "*.png", scale = 1, padding = 0)
MakeAtlas(name = "unitpics_atlas", sourcedir= "unitpics", pattern = "*.dds", scale = 1, padding = 0)

# Notes:

Have the dict return padded coords too! 
make power of two approximation
PNG vs TGA origin